In [1]:
# Common imports and setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)

# Base parameters
n_timesteps = 365  # One year
np.random.seed(42)  # For reproducibility

# Create time index
dates = pd.date_range('2020-01-01', periods=n_timesteps, freq='D')
time_numeric = np.arange(n_timesteps)

# Calendar features (common across all datasets)
day_of_week = dates.dayofweek  # 0=Monday, 6=Sunday
is_weekend = (day_of_week >= 5).astype(int)
month = dates.month
day_of_week_sin = np.sin(2 * np.pi * day_of_week / 7)
day_of_week_cos = np.cos(2 * np.pi * day_of_week / 7)
day_of_month_sin = np.sin(2 * np.pi * dates.day / 30)
day_of_month_cos = np.cos(2 * np.pi * dates.day / 30)
month_sin = np.sin(2 * np.pi * month / 12)
month_cos = np.cos(2 * np.pi * month / 12)

# Common holidays
is_holiday = np.zeros(n_timesteps)
major_holidays = [0, 31, 59, 120, 185, 245, 335, 358]  # Approx: Jan 1, Feb 1, Mar 1, May 1, Jul 5, Sep 2, Dec 1, Dec 24
for day in major_holidays:
    if day < n_timesteps:
        is_holiday[day] = 1

random_holidays = np.random.choice(range(n_timesteps), size=10, replace=False)
is_holiday[random_holidays] = 1

# Function to get temperature with seasonal pattern
def get_temperature(amplitude=10, phase_shift=np.pi, noise_scale=2):
    temp_seasonal = 15 + amplitude * np.sin(2 * np.pi * time_numeric / 365 + phase_shift)
    temp_noise = np.random.randn(n_timesteps) * noise_scale
    return temp_seasonal + temp_noise

# Function to get promotion periods
def get_promotions(num_promotions=3, avg_duration=15):
    is_promotion = np.zeros(n_timesteps)
    promo_starts = np.random.choice(range(n_timesteps - avg_duration), size=num_promotions, replace=False)
    
    for start in promo_starts:
        duration = int(avg_duration * (0.8 + 0.4 * np.random.random()))  # Variable duration
        end = min(start + duration, n_timesteps)
        is_promotion[start:end] = 1
    
    return is_promotion